To improve my understanding and coding skills, I am utilizing python & jupyter notebooks to complete the exercises for Introduction to Mathematical Cryptography.

Needed modules:

In [3]:
import math
import pandas as pd
import numpy as np
from primePy import primes as pr
import csv
import re


Needed constants:

In [60]:
alphabet_string = "abcdefghijklmnopqrstuvwxyz"
#the alphabet by positional index as a dictionary {index: letter}
numeric_alphabet = {}
#the alphabet as a dictionary {letter:index}
alphabet_numeric = {}
for n in range (0,26):
    numeric_alphabet[n] = alphabet_string[n]
    alphabet_numeric[alphabet_string[n]] = n
# a list of the 1000 most common english language words from https://xkcd.com/simplewriter/words.js
with open ('top1000words.txt') as file:
    words_list = [word for line in file for word in line.split("|")]
words_list.append('a')
words_list.append('i')
words_list.append('test')
words_list.append('strung')
words_list.append('separated')
words_list.append('secrets')
words_list.append('shall')
words_list.append('billboard')
words_list.append('think')
#print (words_list)

After completing some of the exercises it became clear that I needed a tool to separate text strings back into their component words. Most distinct English words are less than 12 characters in length with a peak at 3 for word reuse and a peak at 7 for distinct words. 80% of words fall between 4 & 10 letters with the average at 7.6

In [61]:
def word_finder (some_string):
    message = []
    message_words = {}
    strings_to_check = {}
    fragments_to_check = {}
    #this loop sets up strings to check at each index - the potential start to each word.
    for index in range (0, len(some_string)):
        strings_to_check[index] = some_string[index: len(some_string)]
        for index in strings_to_check.keys():
            fragments_at_index =[]
            for word_length in range (14,0, -1):
                possible_fragment_word = strings_to_check[index][0: word_length]
                fragments_at_index.append(possible_fragment_word)
                fragments_to_check [index] = fragments_at_index
    #print(fragments_to_check)
    #this finds all the words at each potential index, descending order of length
    for index in fragments_to_check.keys():
        words_at_index = []
        for fragment in fragments_to_check[index]:
            if fragment in words_list:
                words_at_index.append(fragment)
                continue
            message_words[index]= words_at_index
    #print(message_words)
    #This loop checks to see if there's a word at the next block for each length of word in the words at potential indices. If there's a not a word, then it moves to the next smaller word.
    message_index = -1
    for index in message_words.keys():
        if message_index >= index:
            #print ('index ' +str(index) + ' is smaller than skip '+str(message_index))
            continue
        for word in message_words[index]:
            if index+len(word) >= len (message_words):
                if message_words[index] != message[-1]:
                    message.append(message_words[index][0])
                    break
                break
            if message_words[index+len(word)] != []:
                message_index += len(word)
                message.append (word)
                break
    message_as_string = ' '. join(message)        
    return(message_as_string)



In [46]:
test_string = "thisisatestofallthewordsstrungtogethertobeseparatedintopieces"
test_chunks_strings = word_finder(test_string)
print (test_chunks_strings)

this is a test of all the words strung together to be separated into pieces


In [47]:
#that didn't work on the first example I tried. Why not? 
test2_string = "therearenosecretsbetterthanthesecretsthateverybodyguesses"
testing_2 = word_finder(test2_string)
print(testing_2)
#Because the word "secrets" isn't in the top 1000 words list. I need to update the words library to a longer library for this to work. 

there are no secrets better than the secrets that everybody guesses


Exercises: Chapter 1

1.1 Simple Substitution Ciphers

1.1 (a) Encrypt the following using a rotation of 11 clockwise.  "A page of history is worth a volume of logic."

In [70]:
#clockwise shifts will be defined as positive, counterclockwise as negative
def casear_encryption_wheel (plaintext, shift):
    ciphertext = ''
    #transform the shift into a positive rotation.
    if shift <=0:
        shift =0-shift
    if shift >=26:
        shift =shift%26
    #strip the string of whitespace & punctuation and make it lowercase
    plaintext_new = plaintext.lower()
    plaintext_new = plaintext_new.replace(" ","")
    plaintext_new = re.sub(r'[^\w\s]', '', plaintext_new)
    #translate plaintext to numeric alphabet list
    numeric_plaintext = []
    for char in plaintext_new:
        numeric_plaintext.append(alphabet_numeric[char])
    #shift the letters by the intended cypher shift
    numeric_ciphertext = []
    for value in numeric_plaintext:
        cipher_value = value+shift
        cipher_value = cipher_value%26
        numeric_ciphertext.append(cipher_value)
    #shift the cipher numbers back to letters and a string
    alphabetic_ciphertext = []
    for value in numeric_ciphertext:
        cipher_letter = numeric_alphabet[value]
        cipher_letter_new = cipher_letter.upper()
        alphabetic_ciphertext.append(cipher_letter_new)
    for char in alphabetic_ciphertext:
        ciphertext += char
    return ciphertext

In [68]:
plaintext_1a = "A page of history is worth a volume of logic."
answer_1a = casear_encryption_wheel (plaintext_1a, 11)
print (answer_1a)

LALRPZQSTDEZCJTDHZCESLGZWFXPZQWZRTN


1.1 (b) Decrypt the following, which was encrypted using a rotation of 7 clockwise, "AOLY LHYL UVZL JYLA ZILA ALYA OHUA OLZL JYLA ZAOH ALCL YFIV KFNB LZZL Z"

In [36]:
def casear_decryption_wheel (ciphertext, encipher_shift):
    plaintext = ''
    #remove spaces & punctuation from ciphertext and lowercase for usability
    ciphertext = ciphertext.lower()
    ciphertext = ciphertext.replace(' ', '')
    ciphertext = re.sub(r'[^\w\s]', '', ciphertext)
    #translate ciphertext to numeric list
    numeric_ciphertext = []
    for char in ciphertext:
        numeric_ciphertext.append(alphabet_numeric[char])
    #shift cipher position by negative of enciphering value
    decrypt_shift = (26-encipher_shift)%26
    #apply the decrypt_shift to the numeric_ciphertext
    numeric_plaintext = []
    for num in numeric_ciphertext:
        cipher_num = (num+decrypt_shift)%26
        numeric_plaintext.append(cipher_num)
    #translate it back to letters
    plaintext_letters = []
    for num in numeric_plaintext:
        letter = numeric_alphabet[num]
        plaintext_letters.append(letter)
    #turn it into a string
    for char in plaintext_letters:
        plaintext +=char
    return plaintext

In [38]:
ciphertext_1b = "AOLY LHYL UVZL JYLA ZILA ALYA OHUA OLZL JYLA ZAOH ALCL YFIV KFNB LZZL Z"
answer_1b_without_wordfinder = casear_decryption_wheel (ciphertext_1b, 7)
print(answer_1b_without_wordfinder)
answer_1b = word_finder(answer_1b_without_wordfinder)
print(answer_1b)

therearenosecretsbetterthanthesecretsthateverybodyguesses
there are better than an that hate ate everybody very body guesses


1.1 (c) Decrypt the following message which was encrypted by rotating 1 clockwise for the first letter, then 2 for the second, and so on. "XJHR FTNZ HMZG AHIU ETXZ JNBW NUTR HEPO MDNB JMAU GORF AOIZ OCC"

In [49]:
def rolling_casear_decryption_wheel (ciphertext, encipher_shift, increment_by):
    plaintext = ''
    #remove spaces & punctuation from ciphertext and lowercase for usability
    ciphertext = ciphertext.lower()
    ciphertext = ciphertext.replace(' ', '')
    ciphertext = re.sub(r'[^\w\s]', '', ciphertext)
    #translate ciphertext to numeric list
    numeric_ciphertext = []
    for char in ciphertext:
        numeric_ciphertext.append(alphabet_numeric[char])
    #shift cipher position by negative of enciphering value
    decrypt_shift = (26-encipher_shift)%26
    #apply the decrypt_shift to the numeric_ciphertext
    numeric_plaintext = []
    for num in numeric_ciphertext:
        cipher_num = (num+decrypt_shift)%26
        numeric_plaintext.append(cipher_num)
        decrypt_shift -= increment_by
    #translate it back to letters
    plaintext_letters = []
    for num in numeric_plaintext:
        letter = numeric_alphabet[num]
        plaintext_letters.append(letter)
    #turn it into a string
    for char in plaintext_letters:
        plaintext +=char
    return plaintext

In [51]:
ciphertext_1c = "XJHR FTNZ HMZG AHIU ETXZ JNBW NUTR HEPO MDNB JMAU GORF AOIZ OCC"
answer_1c = rolling_casear_decryption_wheel(ciphertext_1c, 1, 1)
print(answer_1c)
answer_1c = word_finder(answer_1c)
print(answer_1c)

whenangrycounttenbeforeyouspeakifveryangryanhundred
when angry count ten before you speak if very angry an hundred red


1.2 Decrypt each of the following by trying various possible shifts until you arrive at readable text:

In [53]:
# This is a brute force method for the casear cipher, in order to be highly usable it needs a spellchecker to establish which output is correct so that it doesn't spit out 26 of them.
def brutus_casear_wheel (ciphertext):
    all_shifts_plaintext_possible = []
    for shift_number in range (0, 26):
        possible_plaintext = casear_decryption_wheel(ciphertext, shift_number)
        all_shifts_plaintext_possible.append(possible_plaintext)
    return all_shifts_plaintext_possible
# iterate through the common word lengths and check those letter combinations against a dictionary of words
 


1.2 (a) "LWKLQNWKDWLVKDOOQHYHUVHHDELOOERDUGORYHOBDVDWUHH"

In [62]:
ciphertext_2a = 'LWKLQNWKDWLVKDOOQHYHUVHHDELOOERDUGORYHOBDVDWUHH'
answer_2a = brutus_casear_wheel(ciphertext_2a)
for item in answer_2a:
    test_answer = word_finder(item)
    print(item)
    print (test_answer)


lwklqnwkdwlvkdooqhyhuvhhdelooerdugoryhobdvdwuhh

kvjkpmvjcvkujcnnpgxgtuggcdknndqctfnqxgnacucvtgg

juijoluibujtibmmofwfstffbcjmmcpbsempwfmzbtbusff

ithinkthatishallneverseeabillboardlovelyasatree
it that hat is shall never see a billboard as a tree
hsghmjsgzshrgzkkmdudqrddzahkkanzqcknudkxzrzsqdd

grfglirfyrgqfyjjlctcpqccyzgjjzmypbjmtcjwyqyrpcc

fqefkhqexqfpexiikbsbopbbxyfiiylxoailsbivxpxqobb
i i a
epdejgpdwpeodwhhjaranoaawxehhxkwnzhkrahuwowpnaa
a a no a a
docdifocvodncvggizqzmnzzvwdggwjvmygjqzgtvnvomzz

cnbchenbuncmbuffhypylmyyuvcffviulxfipyfsumunlyy

bmabgdmatmblateegxoxklxxtubeeuhtkwehoxertltmkxx

alzafclzslakzsddfwnwjkwwstaddtgsjvdgnwdqsksljww

zkyzebkyrkzjyrccevmvijvvrszccsfriucfmvcprjrkivv

yjxydajxqjyixqbbduluhiuuqrybbreqhtbeluboqiqjhuu

xiwxcziwpixhwpaactktghttpqxaaqdpgsadktanphpigtt
a a
whvwbyhvohwgvozzbsjsfgssopwzzpcofrzcjszmogohfss

vguvaxgungvfunyyarirefrrnovyyobneqybirylnfngerr

uftuzwftmfuetmxxzqhqdeqqmnuxxnamdpxahqxkmemfdqq

testyvesletdslwwypgpcdpplmtwwmzlcowzgpwjldlecpp


ANSWER 1.2 (a) "I think that I shall never see a billboard lovely as a tree."

1.2 (b) "UXENRBWXCUXENFQRLQJUCNABFQNWRCJUCNAJCRXWORWMB"

In [104]:
ciphertext_2b = "UXENRBWXCUXENFQRLQJUCNABFQNWRCJUCNAJCRXWORWMB"
answer_2b = brutus_casear_wheel(ciphertext_2b)
print (answer_2b)

['uxenrbwxcuxenfqrlqjucnabfqnwrcjucnajcrxworwmb', 'twdmqavwbtwdmepqkpitbmzaepmvqbitbmzibqwvnqvla', 'svclpzuvasvcldopjohsalyzdolupahsalyhapvumpukz', 'rubkoytuzrubkcnoingrzkxycnktozgrzkxgzoutlotjy', 'qtajnxstyqtajbmnhmfqyjwxbmjsnyfqyjwfyntsknsix', 'pszimwrsxpszialmglepxivwalirmxepxivexmsrjmrhw', 'oryhlvqrworyhzklfkdowhuvzkhqlwdowhudwlrqilqgv', 'nqxgkupqvnqxgyjkejcnvgtuyjgpkvcnvgtcvkqphkpfu', 'mpwfjtopumpwfxijdibmufstxifojubmufsbujpogjoet', 'loveisnotlovewhichalterswhenitalterationfinds', 'knudhrmnsknudvghbgzksdqrvgdmhszksdqzshnmehmcr', 'jmtcgqlmrjmtcufgafyjrcpqufclgryjrcpyrgmldglbq', 'ilsbfpklqilsbtefzexiqboptebkfqxiqboxqflkcfkap', 'hkraeojkphkrasdeydwhpanosdajepwhpanwpekjbejzo', 'gjqzdnijogjqzrcdxcvgozmnrczidovgozmvodjiadiyn', 'fipycmhinfipyqbcwbufnylmqbyhcnufnyluncihzchxm', 'ehoxblghmehoxpabvatemxklpaxgbmtemxktmbhgybgwl', 'dgnwakfgldgnwozauzsdlwjkozwfalsdlwjslagfxafvk', 'cfmvzjefkcfmvnyztyrckvijnyvezkrckvirkzfewzeuj', 'beluyidejbelumxysxqbjuhimxudyjqbjuhqjyedvydti', 'adktxhcdiadktlwxrw

ANSWER 1.2 (b) "Love is not love which alters when it alteration finds"

1.2 (c) "BGUTBMBGZTFHNLXMKTIPBMAVAXXLXTEPTRLEXTOXKHHFYHKMAXFHNLX"